# MFA input files
Here we will describe the structure of the files required to create and run an INCA model.

In [2]:
import pandas as pd
import pathlib
import ast

The INCA parser does not directly read any files, but takes `pandas dataframes` as input. Thus, the user can make these dataframes using there preferred methods. In the end of this guide, we have a guide to write and read .csv or excel files to import them as correctly formatted dataframes.

The data is validated using the Pandera python package. The validation covers both columns names and data types, thus column names must be specified exactly as shown in these examples. However the dataframes are allowed to contain more columns than the once required, though these extra columns are not parsed to INCA or to the INCA results object. To view the exact validation criteria for a specific data type inspect the data schema's found in the dataschema module.

As an example we will show how to inspect the `model_reactions_schema`, which further described in the section **Model reactions data**

In [3]:
from BFAIR.mfa.INCA.dataschemas import model_reactions_schema
model_reactions_schema.columns

{'rxn_eqn': <Schema Column(name=rxn_eqn, type=DataType(str))>,
 'rxn_id': <Schema Column(name=rxn_id, type=DataType(str))>}

## Model reactions data
This is were the reactions and the atom map in the model are defined. INCA requires the reactions to be defined with arrows `->` for irreversible reactions and `<->` for reversible reactions. There are two different syntaxes for the atom maps. First and simples is just to use letters, e.g. `abc`. If one requires more fine-grained control INCA also supports a syntax which specifies the individual atoms more explicit: `C1:a C2:b C3:c`.

The reaction data requires two columns:
- `rxn_id`: The unique id of the reaction
- `rxn_eqn`: The reaction equation with atom map

In [4]:
model_reactions_example = pd.DataFrame(
    {
        "rxn_id": ["R1", "R2", "R3"],
        "rxn_eqn": ["A (abc) -> B (ab) + D (c)", "B (C1:a C1:b) <-> C (C1:b C2:a)", "C -> D"],
    }
)
model_reactions_example.head()

,rxn_id,rxn_eqn
0,R1,A (abc) -> B (ab) + D (c)
1,R2,B (C1:a C1:b) <-> C (C1:b C2:a)
2,R3,C -> D


Notice that is is allowed to mix the atom mapping syntax and the model may contain reactions without an atom map.

## Tracer data
The tracer data specify the labelled compounds added to the experiment. The dataframe has a row for each experiment-tracer-labelling group combination, more on this later. For most users it will be sufficient to consider each row one experiment-tracer combination. The tracer dataframe has the following required columns:
- `experiment_id`: The experiment in which the tracer was used.
- `met_id`: the metabolite id for labelled compound.
- `tracer_id`: name of the labelled metabolite.
- `atom_ids`: ids of the labelled atoms in the labelled atom group (equivalent to columns of the same name in the INCA GUI)
- `atom_mdv`: mass/isotopomer distribution vector of the labelled atom group (equivalent to columns of the same name in the INCA GUI). The simplest way to use this column is to specify the purity of the labelling group. This is done supplying a list two numbers, e.g. `[0.5, 0.95]` specifies 95% of the compound will be fully labelled in this labelling group. If different atom positions has different purity create a different labelling group for each position. For further description please refer to the INCA manual.

Lets look at a simple and common example.

In [5]:
simple_tracer = pd.DataFrame(
    {
        "experiment_id": ["exp1","exp1"],
        "met_id": ["glc", "glc"],
        "tracer_id": ["[1,2-13C]glucose","[U-13C]glucose"],
        "atom_ids" : [[1,2], [1,2,3,4,5,6]],
        "atom_mdv" : [[0.02, 0.98], [0.001, 0.999]],
        "enrichment" : [0.2, 0.8],
    }
)
simple_tracer.head()

,experiment_id,met_id,tracer_id,atom_ids,atom_mdv,enrichment
0,exp1,glc,"[1,2-13C]glucose","[1, 2]","[0.02, 0.98]",0.2
1,exp1,glc,[U-13C]glucose,"[1, 2, 3, 4, 5, 6]","[0.001, 0.999]",0.8


The specification above specifies a single experiment were a mixture of two types of labelled glucose was used. The enrichment specifies that the labelled medium contained 20% of [1,2-13C]glucose and 80% [U-13C]glucose. The purity of the used tracers (`atom_mdv`) is [1,2-13C]glucose was 98% pure, i.e. 98% of the of the 20% glucose is labelled on the carbon 1 and 2, while 2% of the 20% glucose is has no labels (i.e. not considered the naturally abundant labelling).

### Parallel isotopomer labelling experiment
To specify the a set of parallel isotopomer labelling experiment, i.e. different experiments that which should fitted simultaneously. In the following example, specify the experiment from above in parallel with an experiment conducted with 100% [1-13C]glucose.

In [6]:
parallel_experiment_tracer = pd.DataFrame(
    {
        "experiment_id": ["exp1","exp1","exp2"],
        "met_id": ["glc", "glc", "glc"],
        "tracer_id": ["[1,2-13C]glucose","[U-13C]glucose","[1-13C]glucose"],
        "atom_ids" : [[1,2], [1,2,3,4,5,6],[1]],
        "atom_mdv" : [[0.02, 0.98], [0.001, 0.999],[0.05, 0.95]],
        "enrichment" : [0.2, 0.8,1],
    }
)
parallel_experiment_tracer.head()

,experiment_id,met_id,tracer_id,atom_ids,atom_mdv,enrichment
0,exp1,glc,"[1,2-13C]glucose","[1, 2]","[0.02, 0.98]",0.2
1,exp1,glc,[U-13C]glucose,"[1, 2, 3, 4, 5, 6]","[0.001, 0.999]",0.8
2,exp2,glc,[1-13C]glucose,[1],"[0.05, 0.95]",1.0


### Using labelling groups
The important part for different atom labelling groups is that the `tracer_id` is the same. In the following, we specify that we used a [1,2-13C]glucose tracer where 98% is labelled at carbon atom 1, and the 95% is labelled carbon atom 2. In this case the `enrichment` has to be the same for each labelling group.

In [7]:
two_labelling_groups_tracer = pd.DataFrame(
    {
        "experiment_id": ["exp1","exp1"],
        "met_id": ["glc", "glc"],
        "tracer_id": ["[1,2-13C]glucose","[1,2-13C]glucose"],
        "atom_ids" : [[1], [2]],
        "atom_mdv" : [[0.02, 0.98], [0.05, 0.95]],
        "enrichment" : [1, 1],
    }
)
two_labelling_groups_tracer.head()

,experiment_id,met_id,tracer_id,atom_ids,atom_mdv,enrichment
0,exp1,glc,"[1,2-13C]glucose",[1],"[0.02, 0.98]",1
1,exp1,glc,"[1,2-13C]glucose",[2],"[0.05, 0.95]",1


## Flux measurement data
Flux measurements are typically uptake or secretion rates which this does not require labelling. Therefore this data is also quite simple to define. The required columns are the following:
- `experiment_id` experiment id in which the rate was measured
- `rxn_id`: reaction id
- `flux`: measured/estimated rate typically in mmol/gDW/h
- `flux_std_error`: standard error of the measured/estimated rate

The units has to be consistent for all measurements, because INCA assumes that all rates have the same units. Notice that it is not possible supply a time point for the rate estimates. This is because INCA supports steady state and isotopically non-stationary labelling analysis. Both of these methods assumes that all rates are constant over the time duration and only isotopomer distribution vector are allowed to change over time.

In [8]:
flux_measurements_example = pd.DataFrame(
    {
        "experiment_id": ["exp1", "exp1", "exp1", "exp2", "exp2", "exp2"],
        "rxn_id": ["R1", "R2", "R3", "R1", "R2", "R3"],
        "flux": [1.0, 2.0, 3.0, 1.2, 1.8, 2.8],
        "flux_std_error": [0.1, 0.5, 0.2, 0.1, 0.5, 0.2],
    }
)
flux_measurements_example.head()

,experiment_id,rxn_id,flux,flux_std_error
0,exp1,R1,1.0,0.1
1,exp1,R2,2.0,0.5
2,exp1,R3,3.0,0.2
3,exp2,R1,1.2,0.1
4,exp2,R2,1.8,0.5


## Mass spectrometry measurements
Mass spectrometry measurements are given as isotopomer distribution vectors and the measurement standard error. These can be corrected for natural abundance or not, but by default INCA does the naturel abundance correction, thus this needs to be turned of in the options if it is not required (See XX). The required columns for ms measurements are:
- `experiment_id` experiment id in which the rate was measured
- `met_id`: metabolite id of metabolite which is directly measured or from which the fragment is derived through a derivatization method.
- `ms_id`: id of the measured ms fragment - often multiple fragment can be measured from the same metabolite
- `unlabelled_atoms`: the molecular_formula of the all atoms that cannot be labelled through the introduced labels in the tracers. This typically includes non-carbon elements of the fragment and all elements originating from derivatization agent. INCA uses the unlabelled atoms to correct for natural abundance.
- `idv`: Isotopomer distribution vector in the order lightest to heaviest mass (M0, M1, M2, etc.)
- `idv_std_error`: The standard errors of the idv also in order lightest to heaviest mass (M0, M1, M2, etc.)
- `time`: Time point of measurement only relevant for isotopically non-stationary MFA analysis

In [9]:
ms_measurements_example = pd.DataFrame(
    {
        "experiment_id": ["exp1", "exp1", "exp1", "exp1"],
        "met_id": ["A", "B", "C", "C"],
        "ms_id": ["A_260", "B_381", "C_180", "C_180"],
        "labelled_atom_ids": ["[1,2]", "[C3,C4]", "[3]", "[3]"],
        "unlabelled_atoms": ["C7H19O", "C2H4Si", None, None],
        "idv": [[1.0, 0.4, 0.4], [2.0, 0.2, 0.4], [3.0, 4.0], [1.0, 5.0]],
        "idv_std_error": [[0.1, 0.2, 0.4], [0.2, 0.0001, 0.003], [0.3, 0.4], [0.1, 0.5]],
        "time": [0, 1, 0, 0],
    }
)

ms_measurements_example.head()

,experiment_id,met_id,ms_id,labelled_atom_ids,unlabelled_atoms,idv,idv_std_error,time
0,exp1,A,A_260,"[1,2]",C7H19O,"[1.0, 0.4, 0.4]","[0.1, 0.2, 0.4]",0
1,exp1,B,B_381,"[C3,C4]",C2H4Si,"[2.0, 0.2, 0.4]","[0.2, 0.0001, 0.003]",1
2,exp1,C,C_180,[3],None,"[3.0, 4.0]","[0.3, 0.4]",0
3,exp1,C,C_180,[3],None,"[1.0, 5.0]","[0.1, 0.5]",0


## Note about formatting when reading csv and and excel files
Some of the data inputs requires the element of a cell to be a python list. This can cause issues when reading the data from .csv or excel files. To accommodate the issue simply write a list in python syntax as a string in the csv or excel file. When the file is read using pandas you will evaluate the columns the list strings, which will convert the strings in to python lists.

#### CSV files
For .csv files we can use the convert argument in the `pd.read_csv` functions. Lets use the ms data as an example. The csv file would look as follows

In [10]:
csv_illtration_file = pathlib.Path("../../docs/examples/data/MFA_modelInputsData/ms_measurement_csv_input_example.csv")
with open(csv_illtration_file, "r") as f:
    print(f.read())

experiment_id,met_id,ms_id,labelled_atom_ids,unlabelled_atoms,idv,idv_std_error,time
exp1,A,A_260,"[1,2]",C7H19O,"[1.0, 0.4]","[0.1, 0.2]",0
exp1,B,B_381,"['C3','C4']",C2H4Si,"[2.0]","[0.2]",1


Notice that the list definitions ([]) are inclosed in double quotes, but importantly when the lists contains strings these strings should be also be inclosed in single quotes, e.g. `"['C3','C4']"`. When this csv file is read through `pd.read_csv` the string lists are correctly read.

In [11]:
from_csv = pd.read_csv(csv_illtration_file, converters={"labelled_atom_ids": ast.literal_eval, "idv": ast.literal_eval, "idv_std_error": ast.literal_eval})
from_csv.head()

,experiment_id,met_id,ms_id,labelled_atom_ids,unlabelled_atoms,idv,idv_std_error,time
0,exp1,A,A_260,"[1, 2]",C7H19O,"[1.0, 0.4]","[0.1, 0.2]",0
1,exp1,B,B_381,"[C3, C4]",C2H4Si,[2.0],[0.2],1


We can verify that the data type of the idv is a list.

In [12]:
type(from_csv["idv"][0])

list

And check that the dataframe passes the schema validation

In [13]:
from BFAIR.mfa.INCA.dataschemas import ms_measurements_schema
ms_measurements_schema.validate(from_csv)

,experiment_id,met_id,ms_id,labelled_atom_ids,unlabelled_atoms,idv,idv_std_error,time
0,exp1,A,A_260,"[1, 2]",C7H19O,"[1.0, 0.4]","[0.1, 0.2]",0.0
1,exp1,B,B_381,"[C3, C4]",C2H4Si,[2.0],[0.2],1.0


#### Excel files
The excel file can written more straight forward without double quotes around the lists. For example a cell in labelled_atom_ids can be `['C3','C4']`. Notice, that the atom ids are still wrapped in single quotes because they are defined using the C notation.

Reading from an excel file is slightly more verbose. Here we first read the file and then use `ast.literal_eval()` to convert the strings to python-lists.

In [62]:
excel_illtration_file = pathlib.Path("../../docs/examples/data/MFA_modelInputsData/ms_measurement_csv_input_example.xlsx")
from_excel = pd.read_excel(excel_illtration_file)
from_excel[['labelled_atom_ids', 'idv', 'idv_std_error']] = from_excel[['labelled_atom_ids', 'idv', 'idv_std_error']].applymap(lambda x: ast.literal_eval(x))

Again, we see that the data read from excel passes the schema validation.

In [64]:
ms_measurements_schema.validate(from_excel)

,experiment_id,met_id,ms_id,labelled_atom_ids,unlabelled_atoms,idv,idv_std_error,time
0,exp1,A,A_260,"[1, 2]",C7H19O,"[1.0, 0.4]","[0.1, 0.2]",0.0
1,exp1,B,B_381,"[C3, C4]",C2H4Si,[2.0],[0.2],1.0


Now we have showed the expected structure of the input data you can move on to the next section to read about the how to use the INCA parser.